In [1]:
!pip install pytorchvideo
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for av from https://files.pythonhosted.org/packages/0a/32/56aaa677f0ec369eb68623a2ade90358c1a51b6e3cd6087111bd1e096b82/av-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 33.2 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=c16be96c

In [2]:
import os
import cv2
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

random_seed = 13
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
paths = [
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arson/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Explosion/*',
    '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Fighting/*'
]

In [4]:
normal = glob('/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/*')
abnormal = []
for i in paths:
    abnormal.extend(glob(i))
    
label = [0]*len(normal)+[1]*len(abnormal)
df = pd.DataFrame(zip(normal+abnormal, label), columns=['file', 'label'])
print('Normal: ', len(normal))
print('Abnormal: ', len(abnormal))
df['label'] = df['label'].astype(int)
df.head(), df.tail()

Normal:  150
Abnormal:  400


(                                                file  label
 0  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 1  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 2  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 3  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 4  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0,
                                                   file  label
 545  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 546  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 547  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 548  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 549  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1)

In [5]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.15, shuffle=True,random_state=random_seed)
len(train_df), len(val_df), train_df.head(), val_df.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(467,
 83,
                                                   file  label
 65   /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 113  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 108  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 168  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 236  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1,
                                                   file  label
 0    /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0
 456  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 405  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 380  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      1
 107  /kaggle/input/crimeucfdataset/Anomaly_Dataset/...      0)

In [6]:
train_df.to_csv('train.txt', sep=' ', index=False, header=False)
val_df.to_csv('val.txt', sep=' ', index=False, header=False)

In [7]:
train_lst = list(train_df.itertuples(index=False, name=None))
val_lst = list(val_df.itertuples(index=False, name=None))
print(train_lst[5])

('/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary061_x264.mp4', 1)


In [8]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorch_lightning import LightningDataModule
import torch

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [9]:
video_transform=Compose([
    ApplyTransformToKey(
        key='video',
        transform = Compose([
        UniformTemporalSubsample(64),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        ShortSideScale(60),
        RandomHorizontalFlip(p=0.5)
        ])
    ),
    ApplyTransformToKey(
        key='label',
        transform = Lambda(lambda x: torch.tensor([float(x)]))
    )
])

In [10]:
from torch.utils.data import DataLoader
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        self.BATCH_SIZE = 2
        self.NUM_WORKERS = 2
    
    def train_dataloader(self):
        train_dataset = labeled_video_dataset(
            '/kaggle/working/train.txt',
            clip_sampler=make_clip_sampler('random', 25.6),
            transform=video_transform,
            decode_audio=False
        )
        train_loader = DataLoader(train_dataset, batch_size=self.BATCH_SIZE, num_workers=self.NUM_WORKERS, shuffle=False, pin_memory=True)
        return train_loader
    
    def val_dataloader(self):
        val_dataset = labeled_video_dataset(
            '/kaggle/working/val.txt',
            clip_sampler=make_clip_sampler('random', 25.6),
            transform=video_transform,
            decode_audio=False
        )
        val_loader = DataLoader(val_dataset, batch_size=self.BATCH_SIZE, num_workers=self.NUM_WORKERS, shuffle=False, pin_memory=True)
        return val_loader
    
    def test_dataloader(self):
        test_dataset = labeled_video_dataset(
            '/kaggle/working/val.txt',
            clip_sampler=make_clip_sampler('random', 25.6),
            transform=video_transform,
            decode_audio=False
        )
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False, pin_memory=True)
        return test_loader

In [11]:
# dm = MyDataModule()
# batch=next(iter(dm.val_dataloader()))
# batch.keys()

In [12]:
# batch['video'].shape, batch['label'].shape, batch['video_name'], batch['label']

# Model

In [13]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [14]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', model='efficient_x3d_xs', pretrained=True)

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/efficient_x3d_xs_original_form.pyth" to /root/.cache/torch/hub/checkpoints/efficient_x3d_xs_original_form.pyth
100%|██████████| 14.8M/14.8M [00:00<00:00, 113MB/s] 


In [15]:
class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        vid_mod = torch.hub.load('facebookresearch/pytorchvideo', model='efficient_x3d_xs', pretrained=True)
        self.vid_model = vid_mod
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)
        #parameters
        self.lr=1e-3
        self.batch_size = 2
        #loss function
        self.loss = nn.BCEWithLogitsLoss()
    
    def forward(self, x):
        x = self.vid_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x
        
    def training_step(self, batch, batch_idx):
        y_hat = self(batch['video'])
        loss = self.loss(y_hat, batch['label'])
        self.log("train_loss", loss, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_hat = self(batch['video'])
        loss = self.loss(y_hat, batch['label'])
        self.log("val_loss", loss, sync_dist=True)
        return loss
    
    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.9)
        return {'optimizer':opt, 'lr_scheduler':scheduler}

In [16]:
# m = MyModel()
# out = m(batch['video'])
# print(out.shape, out)
# lab = batch['label']
# print(lab.shape, lab)
# l = m.loss
# print(l(out, lab))

# Training

In [17]:
from pytorch_lightning.loggers import WandbLogger
import wandb
wandb.login(key="1658d378d091cf8659e37004bc727f76b3de8356")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',mode='min')

In [19]:
model = MyModel()
data_module = MyDataModule()
wandb_logger = WandbLogger(log_model='all',project='DL2023_bungno')
trainer = Trainer(devices=2, accelerator='gpu', max_epochs=10, logger = wandb_logger, callbacks=[checkpoint_callback])
trainer.fit(model,data_module)
wandb.finish()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
wandb: Currently logged in as: duongchinh29062003 (chinh). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in ./wandb/run-20240106_091912-ydxevbtm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dutiful-bush-6
wandb: ⭐️ View project at https://wandb.ai/chinh/DL2023_bungno
wandb: 🚀 View run at https://wandb.ai/chinh/DL2023_bungno/runs/ydxevbtm


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [20]:
wandb.finish()